# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# updating Website class to handle dynamic websites
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using Selenium and BeautifulSoup
        """
        self.url = url

        # Set up Selenium with Chrome using WebDriver Manager
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run in headless mode
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            # Open the URL
            driver.get(url)

            # Wait for the dynamic content to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        
        finally:
            # Close the browser
            driver.quit()

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://www.nba.com/game/nyk-vs-orl-0022400419/play-by-play?period=Q1")
print(ed.title)
print(ed.text)

New York Knicks vs Orlando Magic Dec 27, 2024 Play-by-Play | NBA.com
Navigation Toggle
NBA
Games
Home
Tickets
Schedule
Key Dates
2024-25 Regular Season
Emirates NBA Cup Schedule
League Pass Schedule
2024-25 NBA CrunchTime
VR Schedule
National TV Games
Watch
Featured
NBA TV
News
Home
Top Stories
Power Rankings
Transactions
Features
Events
NBA on Christmas Day
Key Dates
Podcasts
Future Starts Now
More
All-Star
NBA Cup
NBA Cup Home
Schedule
Bracket
Cup Tracker
Pick'Em: NBA Cup
Stats
Stats Home
Dunk Score
Players
Teams
Leaders
Stats 101
Cume Stats
Lineups Tool
Media Central Game Stats
Draft
Quick Links
Contact Us
Standings
2024-25 Regular Season Standings
Emirates NBA Cup 2024 Standings
Teams
Atlantic
Boston Celtics
Brooklyn Nets
New York Knicks
Philadelphia 76ers
Toronto Raptors
Central
Chicago Bulls
Cleveland Cavaliers
Detroit Pistons
Indiana Pacers
Milwaukee Bucks
Southeast
Atlanta Hawks
Charlotte Hornets
Miami Heat
Orlando Magic
Washington Wizards
Northwest
Denver Nuggets
Minnesota Tim

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled New York Knicks vs Orlando Magic Dec 27, 2024 Play-by-Play | NBA.com
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Navigation Toggle
NBA
Games
Home
Tickets
Schedule
Key Dates
2024-25 Regular Season
Emirates NBA Cup Schedule
League Pass Schedule
2024-25 NBA CrunchTime
VR Schedule
National TV Games
Watch
Featured
NBA TV
News
Home
Top Stories
Power Rankings
Transactions
Features
Events
NBA on Christmas Day
Key Dates
Podcasts
Future Starts Now
More
All-Star
NBA Cup
NBA Cup Home
Schedule
Bracket
Cup Tracker
Pick'Em: NBA Cup
Stats
Stats Home
Dunk Score
Players
Teams
Leaders
Stats 101
Cume Stats
Lineups Tool
Media Central Game Stats
Draft
Quick Links
Contact Us
Standings
2024-25 Regular Season Standings
Emirates NBA Cup 2024 Standings
Teams
Atlantic
Boston Celtics
Brooklyn Nets
New York Knicks
Philadelphia 76ers
Toronto Raptors
Central


## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, we're starting off with some brain busters, huh? Well, the answer is 4. If you need help with anything more challenging, just let me know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled New York Knicks vs Orlando Magic Dec 27, 2024 Play-by-Play | NBA.com\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nNavigation Toggle\nNBA\nGames\nHome\nTickets\nSchedule\nKey Dates\n2024-25 Regular Season\nEmirates NBA Cup Schedule\nLeague Pass Schedule\n2024-25 NBA CrunchTime\nVR Schedule\nNational TV Games\nWatch\nFeatured\nNBA TV\nNews\nHome\nTop Stories\nPower Rankings\nTransactions\nFeatures\nEvents\nNBA on Christmas Day\nKey Dates\nPodcasts\nFuture Starts Now\nMore\nAll-Star\nNBA Cup\nNBA Cup Home\nSchedule\nBracket\nCup Tracker\nPick'Em: NBA Cup\nStats\nStats Home\nDunk Score\nPlayers\nTeams\nLe

## Time to bring it together - the API for OpenAI is very simple!

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://www.nba.com/game/nyk-vs-orl-0022400419/play-by-play?period=Q1")

"# New York Knicks vs Orlando Magic Game Summary (Dec 27, 2024)\n\nThe webpage provides a detailed play-by-play report of the NBA game played between the New York Knicks and the Orlando Magic on December 27, 2024. The final score was **New York Knicks 108, Orlando Magic 85**.\n\n### Game Highlights\n- The summary includes a chronological list of significant events during the game, including missed shots, rebounds, assists, fouls, and turnovers.\n- The first quarter reported various key plays, including three-pointers by Suggs from the Magic and strategic responses from the Knicks.\n\n### Overall Performance\n- The Knicks showcased strong teamwork, reflected in multiple assists credited to players like Brunson and Bridges.\n- The Magic struggled to maintain control, with multiple turnovers and missed opportunities, particularly highlighted in the first quarter's statistics.\n\n### Additional Notes\n- Game statistics can be further explored through features such as box scores and game ch

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://www.nba.com/game/nyk-vs-orl-0022400419/play-by-play?period=Q1")

# New York Knicks vs Orlando Magic - Game Summary (Dec 27, 2024)

The website provides a detailed play-by-play overview of the NBA game between the New York Knicks and the Orlando Magic, which concluded with the Knicks winning 108 to 85. The breakdown includes a comprehensive sequence of game events, including key plays, points scored, rebounds, turnovers, and fouls throughout the four quarters.

## Game Highlights
- **Final Score:** New York Knicks 108, Orlando Magic 85
- The summary section features a box score and allows users to view further game stats such as QTR breakdowns and additional performance metrics from players.

The comprehensive play-by-play facilitates engagement for fans looking to relive game moments or analyze team performance.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

## Summary of CNN Website

CNN provides a wide range of news and information categorized into various sections including:

- **US News**
- **World News**
- **Politics**
- **Business**
- **Health**
- **Entertainment**
- **Sports**
- **Science**
- **Climate**
  
Additionally, the site features live broadcasting, podcasts, and video content covering current events and breaking news.

### Recent News Highlights:
1. **Train Collision in Florida**: Three firefighters and 12 passengers were injured when a train collided with a fire truck responding to an emergency call.
2. **Security Breach on Delta Flight**: A passenger managed to bypass gate security and board a Delta flight without a ticket.
3. **Weather Alerts**: There are reports of thunderstorms and potential tornadoes causing holiday travel delays from Texas to Alabama.
4. **Fire Incident**: A fire at a Tyson Foods plant in Georgia resulted in 1 death and several serious injuries.
5. **Legal Developments**: Trump requests a pause on the TikTok ban amidst ongoing controversies related to federal policies. 

### Additional Notable Events:
- Mega Millions jackpot worth approximately $1.22 billion was won in California.
- The ongoing Israel-Hamas conflict continues to draw significant international attention, with recent arrests in Gaza.

The CNN website serves as a hub for updated news coverage, ensuring readers have access to important global events as they unfold.

In [20]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI research and safety company based in San Francisco, focusing on developing reliable and beneficial AI systems. Their main product, Claude, is an AI model designed with safety as a top priority.

## Key Highlights

- **Claude AI Models**: 
  - The latest model, **Claude 3.5 Sonnet**, is now available for use.
  - **Claude 3.5 Haiku** was also introduced recently.

- **Developers and Enterprises**:
  - Anthropic offers an API for developers to create AI-powered applications and custom experiences utilizing Claude.

- **Recent Announcements**:
  - *Oct 22, 2024*: Announced new computer use capabilities and updates for Claude 3.5 Sonnet and Haiku.
  - *Sep 4, 2024*: Launched Claude for Enterprise, enhancing features for business applications.
  - *Dec 15, 2022*: Released research on **Constitutional AI**, focusing on safety in AI feedback mechanisms.

## Team and Careers
Anthropic's interdisciplinary team boasts expertise in machine learning, physics, policy, and product development, actively seeking talent to strengthen their workforce.

Overall, Anthropic emphasizes the importance of AI safety and ethical development through rigorous research and innovative product offerings.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [28]:
# Step 0: Scrape website
q1 = Website("https://www.nba.com/game/nyk-vs-orl-0022400419/play-by-play?period=Q1")

# Step 1: Create your prompts

system_prompt = "You are a meticulous and detailed assistant that summarizes that provides detailed play-by-play for nba games in a consumable format. You receive the content of the website and respond in markdown. Ignore text that doesn't belong to the website, like navigation related text."
user_prompt = f"""
    Please provide the minutes played by each player and the players who replaced them, and at what point in the game.
    Game Information: {q1.title}
    Game Summary: {q1.text}
"""

# Step 2: Make the messages list

# o1 model doesn't take system prompts, so we need to comment this out
messages = [
    # {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="o1-mini", messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)

Based on the provided play-by-play data for the **New York Knicks vs. Orlando Magic** game on **December 27, 2024**, I have extracted the minutes played by each player, including substitutions and the specific points in the game when these substitutions occurred. Please note that some assumptions were made due to limited team-specific context in the play-by-play data.

---

### **New York Knicks**

| **Player**        | **Minutes Played** | **Substituted In**    | **Substituted Out**      | **Substitution Time** |
|-------------------|--------------------|-----------------------|--------------------------|------------------------|
| **Brunson**       | 12:00 - 00:18      | Starter                | Subbed out for Payne     | Q1: 00:18              |
| **Suggs**         | 12:00 - 03:03      | Starter                | Subbed out for Harris    | Q1: 03:03              |
| **Anunoby**       | 12:00 - 05:28      | Starter                | Subbed out for Achiuwa   | Q1: 05:28              |
|

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293